# Comprehensive Stock Fundamental Analysis

This notebook performs an in-depth analysis of a stock by examining:
1. Fundamental financial data
2. Key financial ratios and metrics
3. Technical indicators
4. News and geopolitical context (via OpenAI)
5. AI-generated summary report

## Setup and Dependencies

First, let's install and import all required packages. We'll use uv for dependency management.

In [1]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import requests
from datetime import datetime, timedelta
import pytz  # For timezone handling
import json
from dotenv import load_dotenv
import openai

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_theme(style="darkgrid")

# Load environment variables for API keys
load_dotenv()

True

## OpenAI API Configuration

You'll need an OpenAI API key to fetch news summaries and generate reports. Create a `.env` file in the same directory with your API key as `OPENAI_API_KEY=your-key-here`.

In [2]:
# Set up OpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("Warning: OPENAI_API_KEY not found in environment variables.")
try:
    client = openai.OpenAI(api_key=openai_api_key)
    print("OpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")

OpenAI client initialized successfully.


## Input Stock Symbol

Enter the stock symbol you want to analyze.

In [3]:
# Input stock symbol
stock_symbol = "GOOGL" #input("Enter stock symbol (e.g., AAPL, MSFT, GOOGL): ").strip().upper()

# Set analysis timeframes
end_date = datetime.now()

start_date_2y = end_date - timedelta(days=2*365)  # 2 years of data
start_date_1y = end_date - timedelta(days=365)    # 1 year of data

print(f"Analyzing {stock_symbol} from {start_date_2y.date()} to {end_date.date()}")

Analyzing GOOGL from 2023-04-13 to 2025-04-12


## 1. Data Collection

Let's gather historical price data, company information, and financial statements.

In [4]:
def get_stock_data(symbol, start_date, end_date):
    """Fetch historical stock data"""
    try:
        stock = yf.Ticker(symbol)
        # Use tz_localize=None to get timezone-naive datetimes in the result
        data = stock.history(start=start_date, end=end_date, auto_adjust=True)
        data.index = data.index.tz_localize(None)
        if data.empty:
            return None, f"No data found for symbol {symbol}"
        return stock, data
    except Exception as e:
        return None, f"Error fetching data: {e}"

# Fetch stock data
stock, hist_data_2y = get_stock_data(stock_symbol, start_date_2y, end_date)

if isinstance(hist_data_2y, str):
    print(hist_data_2y)  # Print error message
else:
    print(f"Successfully retrieved data for {stock_symbol}")
    hist_data_1y = hist_data_2y[hist_data_2y.index >= start_date_1y]
    
    # Display the stock information
    info = stock.info
    company_name = info.get('longName', stock_symbol)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    
    print(f"Company: {company_name}")
    print(f"Sector: {sector}")
    print(f"Industry: {industry}")
    print(f"Current Price: ${info.get('currentPrice', 'N/A')}")

Successfully retrieved data for GOOGL
Company: Alphabet Inc.
Sector: Communication Services
Industry: Internet Content & Information
Current Price: $157.14


In [5]:
# Get financial data

# Balance Sheet
balance_sheet_annual = stock.balance_sheet
balance_sheet_quarterly = stock.quarterly_balance_sheet

# Income Statement
income_stmt_annual = stock.income_stmt
income_stmt_quarterly = stock.quarterly_income_stmt

# Cash Flow
cash_flow_annual = stock.cashflow
cash_flow_quarterly = stock.quarterly_cashflow

print("Financial data fetched successfully.")

# Display some key financial metrics from the most recent annual data
print("\nRecent Annual Financial Highlights (in millions USD):")
if not income_stmt_annual.empty:
    recent_year = income_stmt_annual.columns[0]
    revenue = income_stmt_annual.loc['Total Revenue', recent_year] / 1e6
    net_income = income_stmt_annual.loc['Net Income', recent_year] / 1e6
    print(f"Revenue: ${revenue:.2f}M")
    print(f"Net Income: ${net_income:.2f}M")
    
    if 'EBITDA' in income_stmt_annual.index:
        ebitda = income_stmt_annual.loc['EBITDA', recent_year] / 1e6
        print(f"EBITDA: ${ebitda:.2f}M")

if not balance_sheet_annual.empty:
    recent_year = balance_sheet_annual.columns[0]
    total_assets = balance_sheet_annual.loc['Total Assets', recent_year] / 1e6
    total_liabilities = balance_sheet_annual.loc['Total Liabilities Net Minority Interest', recent_year] / 1e6
    print(f"Total Assets: ${total_assets:.2f}M")
    print(f"Total Liabilities: ${total_liabilities:.2f}M")
    print(f"Equity: ${(total_assets - total_liabilities):.2f}M")

Financial data fetched successfully.

Recent Annual Financial Highlights (in millions USD):
Revenue: $350018.00M
Net Income: $100118.00M
EBITDA: $135394.00M
Total Assets: $450256.00M
Total Liabilities: $125172.00M
Equity: $325084.00M


## 2. Financial Ratio Analysis

Calculate and visualize key financial ratios that are important for fundamental analysis.

In [6]:
def calculate_financial_ratios(stock, info):
    """Calculate important financial ratios for fundamental analysis"""
    ratios = {}
    
    # Valuation Ratios
    ratios['P/E Ratio'] = info.get('trailingPE', 'N/A')
    ratios['Forward P/E'] = info.get('forwardPE', 'N/A')
    ratios['P/B Ratio'] = info.get('priceToBook', 'N/A')
    ratios['P/S Ratio'] = info.get('priceToSalesTrailing12Months', 'N/A')
    ratios['EV/EBITDA'] = info.get('enterpriseToEbitda', 'N/A')
    
    # Profitability Ratios
    ratios['Gross Margin'] = info.get('grossMargins', 'N/A')
    ratios['Operating Margin'] = info.get('operatingMargins', 'N/A')
    ratios['Net Profit Margin'] = info.get('profitMargins', 'N/A')
    ratios['ROE'] = info.get('returnOnEquity', 'N/A')
    ratios['ROA'] = info.get('returnOnAssets', 'N/A')
    
    # Liquidity Ratios
    ratios['Current Ratio'] = info.get('currentRatio', 'N/A')
    ratios['Quick Ratio'] = info.get('quickRatio', 'N/A')
    
    # Debt Ratios
    ratios['Debt-to-Equity'] = info.get('debtToEquity', 'N/A')
    ratios['Interest Coverage'] = info.get('interestCoverage', 'N/A')
    
    # Dividend Ratios
    ratios['Dividend Yield'] = info.get('dividendYield', 'N/A')
    ratios['Payout Ratio'] = info.get('payoutRatio', 'N/A')
    
    # Growth Rates
    ratios['Revenue Growth (YoY)'] = info.get('revenueGrowth', 'N/A')
    ratios['Earnings Growth (YoY)'] = info.get('earningsGrowth', 'N/A')
    
    return ratios


financial_ratios = calculate_financial_ratios(stock, stock.info)

# Display financial ratios
print("\nKey Financial Ratios:")

ratios_df = pd.DataFrame(list(financial_ratios.items()), columns=['Ratio', 'Value'])
# Convert numeric values and format percentages
ratios_df['Value'] = pd.to_numeric(ratios_df['Value'], errors='ignore')

# Format values for display
formatted_values = []
for idx, row in ratios_df.iterrows():
    if row['Ratio'] in ['Dividend Yield', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'ROE', 'ROA', 'Payout Ratio',
                        'Revenue Growth (YoY)', 'Earnings Growth (YoY)']:
        if isinstance(row['Value'], (int, float)):
            formatted_values.append(f"{row['Value']:.2%}")
        else:
            formatted_values.append(row['Value'])
    else:
        if isinstance(row['Value'], (int, float)):
            formatted_values.append(f"{row['Value']:.2f}")
        else:
            formatted_values.append(row['Value'])
            
ratios_df['Formatted Value'] = formatted_values
display(ratios_df[['Ratio', 'Formatted Value']])


Key Financial Ratios:


/var/folders/w2/1nyptw5136b9mqhj4dvgdky00000gn/T/ipykernel_92932/3060974402.py:45: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  ratios_df['Value'] = pd.to_numeric(ratios_df['Value'], errors='ignore')


,Ratio,Formatted Value
0,P/E Ratio,19.52
1,Forward P/E,17.54
2,P/B Ratio,5.90
3,P/S Ratio,5.51
4,EV/EBITDA,14.27
5,Gross Margin,58.20%
6,Operating Margin,33.97%
7,Net Profit Margin,28.60%
8,ROE,32.91%
9,ROA,16.74%


## 3. Historical Price and Volume Analysis

In [8]:
def safe_plot_display(figure):
    """Safely display a plotly figure, with fallback to a static image if nbformat is missing"""
    try:
        # Try to display the interactive plot
        figure.show()
    except ValueError as e:
        if "nbformat" in str(e):
            # If nbformat is missing or outdated, switch to a non-interactive display
            print("Warning: Interactive plot could not be displayed due to missing or outdated nbformat.")
            print("Displaying static plot instead. Run `!pip install nbformat>=4.2.0` and restart kernel for interactive plots.")
            # Generate a static image
            img_bytes = figure.to_image(format="png")
            from IPython.display import Image, display
            display(Image(img_bytes))
        else:
            # If it's another error, re-raise it
            raise


# Create an interactive price and volume chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05, 
                    subplot_titles=('Price', 'Volume'),
                    row_heights=[0.7, 0.3])

# Price candlestick
fig.add_trace(
    go.Candlestick(
        x=hist_data_2y.index,
        open=hist_data_2y['Open'],
        high=hist_data_2y['High'],
        low=hist_data_2y['Low'],
        close=hist_data_2y['Close'],
        name="Price"
    ),
    row=1, col=1
)

# Add moving averages
hist_data_2y['MA50'] = hist_data_2y['Close'].rolling(window=50).mean()
hist_data_2y['MA200'] = hist_data_2y['Close'].rolling(window=200).mean()

fig.add_trace(
    go.Scatter(
        x=hist_data_2y.index,
        y=hist_data_2y['MA50'],
        name="50-Day MA",
        line=dict(color='orange', width=1)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=hist_data_2y.index,
        y=hist_data_2y['MA200'],
        name="200-Day MA",
        line=dict(color='purple', width=1)
    ),
    row=1, col=1
)

# Volume
colors = ['red' if row['Open'] > row['Close'] else 'green' for i, row in hist_data_2y.iterrows()]
fig.add_trace(
    go.Bar(
        x=hist_data_2y.index,
        y=hist_data_2y['Volume'],
        name="Volume",
        marker_color=colors
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title=f"{stock_symbol} Historical Price and Volume (2 Years)",
    xaxis_rangeslider_visible=False,
    height=800,
    width=1500,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the figure with fallback handling
# safe_plot_display(fig)

## 4. Technical Analysis Indicators

In [10]:
def calculate_technical_indicators(data):
    """Calculate technical indicators for the stock"""
    # Make a copy to avoid modifying the original dataframe
    df = data.copy()
    
    # Moving Averages
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    # Exponential Moving Averages
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    
    # MACD
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    df['BB_Std'] = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
    
    # Average True Range (ATR)
    df['TR1'] = abs(df['High'] - df['Low'])
    df['TR2'] = abs(df['High'] - df['Close'].shift())
    df['TR3'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=14).mean()
    
    return df


# Calculate technical indicators
tech_indicators = calculate_technical_indicators(hist_data_1y)

# Plot the technical indicators
# 1. RSI
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Price with Bollinger Bands', 'RSI', 'MACD'),
                    row_heights=[0.5, 0.25, 0.25])

# Price with Bollinger Bands
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['Close'],
        name="Close Price",
        line=dict(color='blue')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Upper'],
        name="BB Upper",
        line=dict(color='gray', width=1)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Middle'],
        name="BB Middle",
        line=dict(color='gray', width=1, dash='dash')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Lower'],
        name="BB Lower",
        line=dict(color='gray', width=1)
    ),
    row=1, col=1
)

# RSI
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['RSI'],
        name="RSI",
        line=dict(color='purple')
    ),
    row=2, col=1
)

# Add RSI overbought/oversold lines
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=[70] * len(tech_indicators),
        name="Overbought",
        line=dict(color='red', width=1, dash='dash')
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=[30] * len(tech_indicators),
        name="Oversold",
        line=dict(color='green', width=1, dash='dash')
    ),
    row=2, col=1
)

# MACD
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['MACD'],
        name="MACD",
        line=dict(color='blue')
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['MACD_Signal'],
        name="Signal Line",
        line=dict(color='red')
    ),
    row=3, col=1
)

# MACD Histogram
colors = ['green' if val >= 0 else 'red' for val in tech_indicators['MACD_Histogram']]
fig.add_trace(
    go.Bar(
        x=tech_indicators.index,
        y=tech_indicators['MACD_Histogram'],
        name="MACD Histogram",
        marker_color=colors
    ),
    row=3, col=1
)

# Update layout
fig.update_layout(
    title=f"{stock_symbol} Technical Indicators (1 Year)",
    height=900,
    width=1500,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Add y-axis titles
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)

# Show the figure with fallback handling
safe_plot_display(fig)

## 5. News and Geopolitical Analysis via OpenAI

Let's fetch and analyze recent news about the company using OpenAI.

In [ ]:
def get_news_analysis(company_name, stock_symbol, client):
    """Get news analysis from OpenAI"""
    if client is None:
        return "OpenAI client not initialized. Please set up your API key."
    
    try:
        # Current date for context
        today = datetime.now().strftime("%Y-%m-%d")
        
        prompt = f"""
        Today is {today}. Provide a comprehensive summary of the most significant recent news about {company_name} ({stock_symbol}) 
        that could impact its stock price and fundamental value. Focus on:
        
        1. Recent earnings reports and financial performance
        2. Major business developments (new products, services, markets)
        3. Leadership changes or organizational restructuring
        4. Regulatory developments affecting the company
        5. Macroeconomic factors influencing the company and its industry
        6. Geopolitical events that might impact operations or supply chains
        7. Competitive landscape changes
        
        Format your response as a well-structured analysis with clear sections and bullet points where appropriate.
        Include dates of key events where possible. Highlight the potential impact of each development on the company's future performance.
        """
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Use appropriate model based on your OpenAI subscription
            messages=[
                {"role": "system", "content": "You are a financial analyst with expertise in stock market analysis and economics."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1500
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error fetching news analysis: {e}"


company_name = stock.info.get('longName', stock_symbol)
news_analysis = get_news_analysis(company_name, stock_symbol, client)
print(f"\nNews and Geopolitical Analysis for {company_name} ({stock_symbol}):\n")
print(news_analysis)


News and Geopolitical Analysis for Alphabet Inc. (GOOGL):

### Comprehensive Summary of Recent News About Alphabet Inc. (GOOGL)

As of April 12, 2025, Alphabet Inc. (GOOGL) has experienced several significant developments that could impact its stock price and fundamental value. Below is a structured analysis of these developments:

#### 1. Recent Earnings Reports and Financial Performance
- **Q1 2025 Earnings Report (Released April 10, 2025)**:
  - **Revenue**: $75 billion, a 12% increase year-over-year.
  - **Net Income**: $18 billion, up from $16 billion in Q1 2024.
  - **Key Drivers**: Growth in advertising revenue, particularly from YouTube and Google Cloud services.
  - **Impact**: Strong earnings performance suggests robust demand for digital advertising and cloud solutions, potentially boosting investor confidence and stock price.

#### 2. Major Business Developments
- **Launch of Bard AI 2.0 (March 2025)**:
  - Enhanced capabilities in natural language processing and integrati

## 6. Comprehensive AI-Generated Analysis Report

Now, let's generate a comprehensive report that analyzes all the data we've collected.

In [14]:
def generate_comprehensive_report(stock_symbol, stock_info, financial_ratios, tech_indicators, news_analysis, client):
    """Generate a comprehensive stock analysis report using OpenAI"""
    if client is None:
        return "OpenAI client not initialized. Please set up your API key."
    
    try:
        # Prepare financial ratios string
        ratios_str = "\n".join([f"{ratio}: {value}" for ratio, value in financial_ratios.items()])
        
        # Get key technical indicators values for the most recent date
        recent_tech = {}
        if tech_indicators is not None and not tech_indicators.empty:
            recent_date = tech_indicators.index[-1]
            recent_tech = {
                'Price': tech_indicators.loc[recent_date, 'Close'],
                'RSI': tech_indicators.loc[recent_date, 'RSI'],
                'MACD': tech_indicators.loc[recent_date, 'MACD'],
                'MACD Signal': tech_indicators.loc[recent_date, 'MACD_Signal'],
                'ATR': tech_indicators.loc[recent_date, 'ATR'],
                'BB Upper': tech_indicators.loc[recent_date, 'BB_Upper'],
                'BB Middle': tech_indicators.loc[recent_date, 'BB_Middle'],
                'BB Lower': tech_indicators.loc[recent_date, 'BB_Lower']
            }
        
        tech_str = "\n".join([f"{indicator}: {value}" for indicator, value in recent_tech.items()])
        
        # Prepare info summary
        info_summary = f"""
        Company: {stock_info.get('longName', stock_symbol)}
        Symbol: {stock_symbol}
        Sector: {stock_info.get('sector', 'N/A')}
        Industry: {stock_info.get('industry', 'N/A')}
        Current Price: ${stock_info.get('currentPrice', 'N/A')}
        52-Week Range: ${stock_info.get('fiftyTwoWeekLow', 'N/A')} - ${stock_info.get('fiftyTwoWeekHigh', 'N/A')}
        Market Cap: ${stock_info.get('marketCap', 'N/A')}
        Beta: {stock_info.get('beta', 'N/A')}
        """
        
        # Current date for context
        today = datetime.now().strftime("%Y-%m-%d")
        
        prompt = f"""
        Today is {today}. As a financial analyst, provide a comprehensive investment analysis report for {stock_symbol} based on the following information:
        
        COMPANY INFORMATION:
        {info_summary}
        
        FINANCIAL RATIOS:
        {ratios_str}
        
        TECHNICAL INDICATORS (Most Recent):
        {tech_str}
        
        NEWS ANALYSIS:
        {news_analysis}
        
        Based on all this information, provide a comprehensive investment analysis with the following sections:
        
        1. Executive Summary - A brief overview of the company and its current situation
        2. Fundamental Analysis - Analysis of financial health, valuation, and growth prospects
        3. Technical Analysis - Interpretation of price movements and technical indicators
        4. News Impact Analysis - How recent news affects the company's prospects
        5. Risk Assessment - Key risks facing the company
        6. Investment Outlook - Overall assessment including strengths, weaknesses, opportunities, and threats
        7. Recommendation - Clear investment recommendation (Buy/Hold/Sell) with reasoning
        
        The report should be well-structured with clear sections and bullet points where appropriate.
        Focus on providing actionable insights based on the data provided.
        """
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Use appropriate model based on your OpenAI subscription
            messages=[
                {"role": "system", "content": "You are a senior financial analyst with extensive experience in equity research and investment analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2500
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating comprehensive report: {e}"


print("\nGenerating comprehensive investment analysis report...\n")
comprehensive_report = generate_comprehensive_report(
    stock_symbol,
    stock.info,
    financial_ratios,
    tech_indicators,
    news_analysis,
    client
)
print(comprehensive_report)


Generating comprehensive investment analysis report...

# Investment Analysis Report for Alphabet Inc. (GOOGL)

## 1. Executive Summary
Alphabet Inc. (GOOGL) is a leading player in the Communication Services sector, primarily known for its dominance in internet content and information. As of April 12, 2025, GOOGL's stock is trading at $157.14, with a market capitalization of approximately $1.93 trillion. The company has shown resilience in its financial performance, highlighted by a recent Q1 2025 earnings report that demonstrated strong revenue growth driven by advertising and cloud services. However, Alphabet faces challenges from regulatory scrutiny and increased competition, which could impact its future growth trajectory.

## 2. Fundamental Analysis
### Financial Health
- **Revenue**: $75 billion in Q1 2025, a 12% increase YoY.
- **Net Income**: $18 billion, reflecting a solid growth from $16 billion in Q1 2024.
- **Margins**: 
  - Gross Margin: 58.2%
  - Operating Margin: 33.97%

## 7. Export Report to PDF (Optional)

You can save the comprehensive report to a PDF file for sharing or future reference.

In [13]:
# Uncomment and run this if you want to save the report as an HTML file
# (You'd need to install additional libraries for PDF export)

'''
if 'comprehensive_report' in locals():
    from IPython.display import HTML
    
    report_title = f"{stock_symbol} Investment Analysis Report - {datetime.now().strftime('%Y-%m-%d')}"
    
    # Format HTML
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>{report_title}</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; margin: 40px; }}
            h1 {{ color: #2c3e50; }}
            h2 {{ color: #3498db; border-bottom: 1px solid #eee; padding-bottom: 10px; }}
            .report-date {{ color: #7f8c8d; font-style: italic; margin-bottom: 30px; }}
            .disclaimer {{ font-size: 0.8em; color: #7f8c8d; margin-top: 50px; border-top: 1px solid #eee; padding-top: 20px; }}
        </style>
    </head>
    <body>
        <h1>{report_title}</h1>
        <p class="report-date">Generated on {datetime.now().strftime('%B %d, %Y')}</p>
        
        <div class="report-content">
            {comprehensive_report.replace('\n', '<br>')}
        </div>
        
        <p class="disclaimer">
            Disclaimer: This report is generated for informational purposes only and does not constitute investment advice. 
            The analysis is based on historical data and AI-generated insights, which may not accurately predict future performance. 
            Always conduct your own research before making investment decisions.
        </p>
    </body>
    </html>
    """
    
    # Save HTML file
    report_filename = f"{stock_symbol}_Investment_Analysis_{datetime.now().strftime('%Y%m%d')}.html"
    with open(report_filename, 'w') as f:
        f.write(html_content)
    
    print(f"Report saved as {report_filename}")
    
    # Display link to open the file
    display(HTML(f'<a href="{report_filename}" target="_blank">Open Investment Report</a>'))
'''

'\nif \'comprehensive_report\' in locals():\n    from IPython.display import HTML\n    \n    report_title = f"{stock_symbol} Investment Analysis Report - {datetime.now().strftime(\'%Y-%m-%d\')}"\n    \n    # Format HTML\n    html_content = f"""\n    <!DOCTYPE html>\n    <html>\n    <head>\n        <title>{report_title}</title>\n        <style>\n            body {{ font-family: Arial, sans-serif; line-height: 1.6; margin: 40px; }}\n            h1 {{ color: #2c3e50; }}\n            h2 {{ color: #3498db; border-bottom: 1px solid #eee; padding-bottom: 10px; }}\n            .report-date {{ color: #7f8c8d; font-style: italic; margin-bottom: 30px; }}\n            .disclaimer {{ font-size: 0.8em; color: #7f8c8d; margin-top: 50px; border-top: 1px solid #eee; padding-top: 20px; }}\n        </style>\n    </head>\n    <body>\n        <h1>{report_title}</h1>\n        <p class="report-date">Generated on {datetime.now().strftime(\'%B %d, %Y\')}</p>\n        \n        <div class="report-content">\n   

## 8. Conclusion

This notebook provides a comprehensive stock analysis framework that combines:

1. Historical price and volume data
2. Financial fundamentals and ratio analysis
3. Technical indicators
4. News and geopolitical context via OpenAI
5. AI-generated comprehensive investment analysis

To use this notebook effectively:
1. Make sure you have set up your OpenAI API key in a `.env` file
2. Enter a valid stock symbol when prompted
3. Review each section of the analysis
4. Use the comprehensive report as a starting point for your investment research

Remember that all investment decisions should incorporate additional research and consider your personal financial situation and risk tolerance.